# Data Analysis

In [1]:
import boto3
import pandas as pd
import io
import json

In [2]:
bucket_name = 'rearc-quest-bls'
bls_file_name = 'pr.data.0.Current'
data_file_name = 'datausa-results.json'

## Load BLS File

In [3]:
s3 = boto3.client("s3")
obj = s3.get_object(Bucket=bucket_name, Key=bls_file_name)
data = obj["Body"].read().decode('utf-8')
print(data[:500])

series_id        	year	period	       value	footnote_codes
PRS30006011      	1995	Q01	         2.6	
PRS30006011      	1995	Q02	         2.1	
PRS30006011      	1995	Q03	         0.9	
PRS30006011      	1995	Q04	         0.1	
PRS30006011      	1995	Q05	         1.4	
PRS30006011      	1996	Q01	        -0.2	
PRS30006011      	1996	Q02	        -0.3	
PRS30006011      	1996	Q03	        -0.1	
PRS30006011      	1996	Q04	         0.2	
PRS30006011      	1996	Q05	        -0.1	
PRS30006011      	199


In [4]:
bls_df = pd.read_csv(io.StringIO(data), sep='\t')
bls_df.columns = [col.strip() for col in bls_df.columns]
bls_df = bls_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
print(bls_df.head())
print(bls_df.dtypes)
print(bls_df.columns)

     series_id  year period  value footnote_codes
0  PRS30006011  1995    Q01    2.6            NaN
1  PRS30006011  1995    Q02    2.1            NaN
2  PRS30006011  1995    Q03    0.9            NaN
3  PRS30006011  1995    Q04    0.1            NaN
4  PRS30006011  1995    Q05    1.4            NaN
series_id          object
year                int64
period             object
value             float64
footnote_codes     object
dtype: object
Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object')


## Load API File

In [5]:
obj = s3.get_object(Bucket=bucket_name, Key=data_file_name)
data = obj["Body"].read().decode('utf-8')
json_data = json.loads(data)
print(json_data)
json_data.keys()

{'annotations': {'dataset_link': 'http://www.census.gov/programs-surveys/acs/', 'source_description': 'The American Community Survey (ACS) is conducted by the US Census and sent to a portion of the population every year.', 'source_name': 'Census Bureau', 'table_id': 'B01003', 'subtopic': 'Demographics', 'topic': 'Diversity', 'dataset_name': 'ACS 1-year Estimate'}, 'page': {'limit': 0, 'offset': 0, 'total': 10}, 'columns': ['Nation ID', 'Nation', 'Year', 'Population'], 'data': [{'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2013, 'Population': 316128839.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2014, 'Population': 318857056.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2015, 'Population': 321418821.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2016, 'Population': 323127515.0}, {'Nation ID': '01000US', 'Nation': 'United States', 'Year': 2017, 'Population': 325719178.0}, {'Nation ID': '01000US', 'Nation': 'United State

dict_keys(['annotations', 'page', 'columns', 'data'])

In [6]:
data_json = json_data['data']
data_json = json.dumps(data_json)
data_df = pd.read_json(io.StringIO(data_json))
print(data_df)
print(data_df.dtypes)
print(data_df.columns)

  Nation ID         Nation  Year  Population
0   01000US  United States  2013   316128839
1   01000US  United States  2014   318857056
2   01000US  United States  2015   321418821
3   01000US  United States  2016   323127515
4   01000US  United States  2017   325719178
5   01000US  United States  2018   327167439
6   01000US  United States  2019   328239523
7   01000US  United States  2021   331893745
8   01000US  United States  2022   333287562
9   01000US  United States  2023   334914896
Nation ID     object
Nation        object
Year           int64
Population     int64
dtype: object
Index(['Nation ID', 'Nation', 'Year', 'Population'], dtype='object')


### DataUSA API Data: Mean and Standard Deviation of the Population from Years 2013-2018 

In [7]:
std_data = round(data_df[data_df['Year'].between(2013,2018)]['Population'].std(),4)
mean_data = round(data_df[data_df['Year'].between(2013,2018)]['Population'].mean(),4)

print('Standard Deviation of Population from 2013-2018:', std_data)
print('Mean of Population from 2013-2018:', mean_data)

Standard Deviation of Population from 2013-2018: 4158441.0409
Mean of Population from 2013-2018: 322069808.0


### BLS data: Find the best year of each series

In [8]:
bls_grouped = bls_df.groupby(['series_id','year'])['value'].sum().reset_index()
bls_best_idx = bls_grouped.groupby("series_id")["value"].idxmax()
df_bls_best = bls_grouped.loc[bls_best_idx, ["series_id", "year", "value"]].reset_index()
df_bls_best.head()

,index,series_id,year,value
0,27,PRS30006011,2022,20.500
1,58,PRS30006012,2022,17.100
2,65,PRS30006013,1998,705.895
3,108,PRS30006021,2010,17.700
4,139,PRS30006022,2010,12.400


### Population Data for PRS30006032, Q01

In [9]:
df_bls_q01 = bls_df[(bls_df['series_id']=='PRS30006032') & (bls_df['period']=='Q01')]
df_data_select = data_df[['Year','Population']]
df_results = df_bls_q01.merge(df_data_select, how='left', left_on='year', right_on='Year').drop(['Year','footnote_codes'], axis=1)
df_results

,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.2,NaN
2,PRS30006032,1997,Q01,2.8,NaN
3,PRS30006032,1998,Q01,0.9,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.5,NaN
6,PRS30006032,2001,Q01,-6.3,NaN
7,PRS30006032,2002,Q01,-6.6,NaN
8,PRS30006032,2003,Q01,-5.7,NaN
9,PRS30006032,2004,Q01,2.0,NaN
